# Module 2: Speed ML Development with Apache Iceberg offline store compaction

Feature Store now supports Apache Iceberg as a table format for storing features. This accelerates model development by enabling faster query performance when extracting ML training datasets, taking advantage of Iceberg table compaction. Depending on the design of your feature groups and their scale, you can experience training query performance improvements of 10x to 100x by using this new capability.

---

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Create Feature Groups in Iceberg table format](#Create-Feature-Groups-in-Iceberg-table-format)
1. [Compaction](#Compaction)
   1. [Compaction using Athena](#Compaction-using-Athena)
   1. [Compaction using Spark](#Compaction-using-Spark)
1. [Scheduled Compaction](#Scheduled-Compaction)

# Background

In this notebook, you will learn:
* how to create feature groups in Offline Store using Apache Iceberg format
* how to schedule table maintenance operations (compaction, storage reclaim) using Amazon Athena and AWS Glue



[Apache Iceberg](https://iceberg.apache.org/) is an open table format for very large analytic datasets. It manages large collections of files as tables, and it supports modern analytical data lake operations such as record-level insert, update, delete, and time travel queries. 

With Feature Store, you can create feature groups with Iceberg table format as an alternative to the default standard Glue format.  With that, customers can leverage the table format to use Iceberg’s file compaction and data pruning features as per their use case and optimization requirements. Iceberg also enables customers to perform deletion, time-travel queries, high concurrent transactions, and higher performance of selective queries.

By combining Iceberg as a table format and table maintenance operations such as compaction, customer will benefit from faster query performance when working with offline feature groups at scale and as a result help customer build training dataset faster. 

The following diagram shows the structure of the offline store using Iceberg as a table format.

![Offline File Compaction](../images/smfs_apache_iceberg_compaction.png "Offline File Compaction")

In this notebook, you will learn how to create feature groups using the Iceberg format, execute Iceberg’s table management procedures using AWS Athena and use AWS services to schedule these tasks to run on-demand or on a schedule.

# Setup

#### Imports

In [ ]:
import boto3
import pandas as pd
import numpy as np
import time
from time import gmtime, strftime, sleep
from random import randint
import subprocess
import sagemaker
from sagemaker.session import Session
from sagemaker import get_execution_role
from sagemaker.feature_store.feature_group import FeatureGroup
import importlib
import logging
import sys
import os
import io

In [ ]:
current_sagemaker_release = '2.132.0'
curr_major, curr_minor, curr_patch = current_sagemaker_release.split('.')
sagemaker_version = sagemaker.__version__
sm_major, sm_minor, sm_patch = sagemaker_version.split('.')
if int(sm_major) < int(curr_major) or int(sm_minor) < int(curr_minor):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', f'sagemaker=={current_sagemaker_release}'])
    importlib.reload(sagemaker)

In [ ]:
logger = logging.getLogger('__name__')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

In [ ]:
logger.info(f'Using SageMaker version: {sagemaker.__version__}')
logger.info(f'Using Pandas version: {pd.__version__}')

#### Essentials

In [ ]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
default_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
logger.info(f'Default S3 bucket = {default_bucket}')
prefix = 'sagemaker-feature-store'
# check if need this
s3_bucket_name = sagemaker_session.default_bucket()

featurestore_runtime_client = sagemaker_session.boto_session.client('sagemaker-featurestore-runtime', region_name=region)

boto_session = boto3.Session(region_name=region)
sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)
feature_store_session = sagemaker.Session(boto_session=boto_session, 
                                          sagemaker_client=sagemaker_client, 
                                          sagemaker_featurestore_runtime_client=featurestore_runtime_client)

#### Load datasets

In [ ]:
order_data = pd.read_csv("../data/transformed/orders.csv")

In [ ]:
order_data['order_id'] = order_data['order_id'].astype('string')
order_data['event_time'] = order_data['event_time'].astype('string')
order_data['customer_id'] = order_data['customer_id'].astype('string')
order_data['product_id'] = order_data['product_id'].astype('string')

In [ ]:
order_data

# Create Feature Groups in Iceberg table format

In [ ]:
# define feature group name
orders_feature_group_iceberg_name = "orders-feature-group-iceberg-" + strftime("%d-%H-%M-%S", gmtime())

In [ ]:
orders_feature_group_iceberg = FeatureGroup(
    name=orders_feature_group_iceberg_name, sagemaker_session=sagemaker_session
)

In [ ]:
orders_feature_group_iceberg.load_feature_definitions(data_frame=order_data)

Select Iceberg as a table format when creating new feature groups. A new optional parameter TableFormat can be configured either interactively using the Amazon SageMaker Studio, or through code using the API or the SDK. This parameter can accept the values ICEBERG, or GLUE for current Glue format.  The following code snippet shows you how to create a feature group using the Iceberg format and FeatureGroup.create API of the SageMaker SDK.

In [ ]:
from sagemaker.feature_store.inputs import TableFormatEnum

record_identifier_order_feature_name = "order_id"

orders_feature_group_iceberg.create(
    s3_uri=f"s3://{s3_bucket_name}/{prefix}",
    record_identifier_name=record_identifier_order_feature_name,
    event_time_feature_name="event_time",
    role_arn=role,
    enable_online_store=True,
    table_format=TableFormatEnum.ICEBERG
)

In [ ]:
orders_feature_group_iceberg.describe()

In [ ]:
orders_fg = FeatureGroup(name=orders_feature_group_iceberg_name, sagemaker_session=feature_store_session)

#### Ingest Data

In [ ]:
def wait_for_feature_group_creation_complete(feature_group):
    status = feature_group.describe().get('FeatureGroupStatus')
    print(f'Initial status: {status}')
    while status == 'Creating':
        logger.info(f'Waiting for feature group: {feature_group.name} to be created ...')
        time.sleep(5)
        status = feature_group.describe().get('FeatureGroupStatus')
    if status != 'Created':
        raise SystemExit(f'Failed to create feature group {feature_group.name}: {status}')
    logger.info(f'FeatureGroup {feature_group.name} was successfully created.')

In [ ]:
wait_for_feature_group_creation_complete(orders_fg)

In [ ]:
%%time
logger.info(f'Ingesting data into feature group: {orders_fg.name} ...')
orders_fg.ingest(data_frame=order_data, max_processes=16, wait=True)
logger.info(f'{len(order_data)} orders records ingested into feature group: {orders_fg.name}')

#### Query Data

In [ ]:
orders_query = orders_fg.athena_query()
orders_table = orders_query.table_name
orders_database = orders_query.database

In [ ]:
query_string = f'SELECT * ' \
    f'FROM' \
    f'    (SELECT *,' \
    f'         row_number()' \
    f'        OVER (PARTITION BY "order_id"' \
    f'    ORDER BY  event_time desc, Api_Invocation_Time DESC, write_time DESC) AS row_num' \
    f'    FROM "{orders_database}"."{orders_table}")' \
    f'WHERE row_num = 1 and NOT is_deleted limit 10000;'

%store query_string
query_string

In [ ]:
query_results= 'sagemaker-featurestore/athena-results'
output_location = f's3://{default_bucket}/{query_results}/query_results/'
print(f'Athena query output location: \n{output_location}')

In [ ]:
%%timeit
orders_query.run(query_string=query_string, output_location=output_location)
orders_query.wait()

In [ ]:
joined_df = orders_query.as_dataframe()
joined_df.head()

# Compaction

## Compaction using Athena

[Amazon Athena](https://aws.amazon.com/athena/) is a serverless SQL query engine that natively supports Iceberg management procedures. In this section, you will use Athena to manually compact the offline feature group you created.  

Note you will need to use [Athena engine version 3](https://docs.aws.amazon.com/athena/latest/ug/engine-versions-reference-0003.html). For this, you can create a new workgroup, or configure an existing workgroup, and select the recommended Athena engine version 3. For more information and instructions for changing your Athena engine version, refer to Changing Athena engine versions.

![Offline File Compaction](../images/smfs_athena_workgroup.png "Offline File Compaction")

#### Query pre compaction

The $partitions table provides a detailed overview of the partitions of the Iceberg table. You can retrieve the information about the partitions of the Iceberg table. Let's have a look at the number of files and total bytes size (pre-compaction).

In [ ]:
stats_query_string = f'SELECT sum(total_size), sum(file_count) ' \
    f'FROM "{orders_table}$partitions"' \

%store stats_query_string
stats_query_string

In [ ]:
%%timeit
orders_query.run(query_string=stats_query_string, output_location=output_location)
orders_query.wait()

In [ ]:
stats_joined_df = orders_query.as_dataframe()
stats_joined_df.head()

#### Iceberg Compaction Procedures

As data accumulates into an Iceberg table, queries gradually become less efficient because of the increased processing time required to open additional files. Compactions optimizes the structural layout of the table without altering table content. 

For this, you will use the `OPTIMIZE table REWRITE DATA` compaction table maintenance command in Athena. The following syntax shows how to optimize data layout of a Feature Group stored using the Iceberg table format. See [documentation](https://docs.aws.amazon.com/athena/latest/ug/querying-iceberg-data-optimization.html) for more detail.

```sql
OPTIMIZE "sagemaker_featurestore"."<FEATURE_GROUP_TABLE_NAME>" REWRITE DATA USING BIN_PACK
```

<div class="alert alert-info"> 💡 <strong> Iceberg Optimize more Runs needed: </strong>
In the event the table could not be compacted within the quey execution timeout for Athena, you will receive an error ICEBERG_OPTIMIZE_MORE_RUNS_NEEDED. This happens if the entire table could not be compacted within the execution time and you should rerun the command, to resume compaction for remaining partitions.
</div>

![Offline File Compaction](../images/smfs_optimize_query_more_runs_needed.png "Offline File Compaction")

After running the optimize command, you will use the `VACUUM` procedure which performs snapshot expiration and removal of orphan files. These actions reduce metadata size and remove files not in the current table state that are also older than the retention period specified for the table. See [documentation](https://docs.aws.amazon.com/athena/latest/ug/querying-iceberg-data-optimization.html) for more detail.

```sql
VACUUM "sagemaker_featurestore"."<FEATURE_GROUP_TABLE_NAME>"
```

We created a script to run the procedures in Python. Let's explore it.

* The script executes the OPTIMIZE query using boto3.
* It handles the error message error ICEBERG_OPTIMIZE_MORE_RUNS_NEEDED which requires you to resume compaction for remaining partitions.
* It executes VACUUM query if the OPTIMIZE query is successful.

In [ ]:
!pip3 install pyspark
!pip3 install git+https://github.com/awslabs/aws-glue-libs.git

In [ ]:
!pygmentize ./smfs_offline_compaction.py

#### Execute the compaction procedures

In [ ]:
!python smfs_offline_compaction.py --region {region} --database {orders_database} --table {orders_table} --workgroup primary --outputlocation {output_location}

#### Query post compaction

In [ ]:
%%timeit
orders_query.run(query_string=query_string, output_location=output_location)
orders_query.wait()

In [ ]:
joined_df = orders_query.as_dataframe()
joined_df.head()

Depending on the design of your feature groups and their scale, you can experience training query performance improvements of 10x to 100x by using this new capability. Since the dataset used in this example is small, you might not see a noticeable improvement in query performance. Let's have a look at the number of files and total bytes size post compaction.

In [ ]:
%%timeit
orders_query.run(query_string=stats_query_string, output_location=output_location)
orders_query.wait()

In [ ]:
stats_joined_df = orders_query.as_dataframe()
stats_joined_df.head()

## Compaction using Spark

Customers can also use Spark to manage the compaction jobs and maintenance methods. For more detail on the Spark procedures, see the [documentation](https://iceberg.apache.org/docs/latest/spark-procedures/).

You first need to configure some of the common properties.

``` sql
%%configure -f
{
  "conf": {
    "spark.sql.catalog.smfs": "org.apache.iceberg.spark.SparkCatalog",
    "spark.sql.catalog.smfs.catalog-impl": "org.apache.iceberg.aws.glue.GlueCatalog",
    "spark.sql.catalog.smfs.warehouse": "<YOUR_ICEBERG_DATA_S3_LOCATION>",
    "spark.sql.extensions":"org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
    "spark.sql.catalog.smfs.glue.skip-name-validation": "true"
  }
}
```

The following code can be used to optimize the feature groups via Spark.

``` sql
spark.sql(f"""CALL smfs.system.rewrite_data_files(table => '{DATABASE}.`{ICEBERG_TABLE}`')""")
```

You can then execute the next two table maintenance procedures to remove older snapshots and orphan files which are no longer needed.

``` sql
spark.sql(f"""CALL smfs.system.expire_snapshots(table => '{DATABASE}.`{ICEBERG_TABLE}`', older_than => TIMESTAMP '{one_day_ago}', retain_last => 1)""")
spark.sql(f"""CALL smfs.system.remove_orphan_files(table => '{DATABASE}.`{ICEBERG_TABLE}`')""")
```

You can then incorporate the above Spark commands into your Spark environment. For example, you can create a job that performs the optimization above on a desired schedule or in a pipeline after ingestion. 


# Scheduled Compaction

In this section, you will learn how to automate the table management procedures to compact your offline feature store. The diagram below illustrates the architecture for creating Feature Groups in Iceberg table format and a fully automated table management solution which includes file compaction and cleanup operations. 

![Offline File Compaction](../images/smfs_scheduled_compaction.png "Offline File Compaction")

* At a high level, you created a Feature Group using the Iceberg table format and ingested records into the online feature store in the previous section. 
* Feature values were automatically replicated from the online store to the historical offline store. 
* [Athena](https://aws.amazon.com/athena/) will be used to run the Iceberg management procedures.
* To schedule the procedures, you will setup a [AWS Glue](https://aws.amazon.com/glue/) job using a Python shell script and create an AWS Glue job schedule. 

### AWS Glue Job Setup

#### Create IAM Role

First you need to create a role for AWS Glue to have permissions to access Amazon Athena, Amazon S3, and CloudWatch.
* Navigate to the IAM console.
* Create an AWS Identity and Access Management (IAM) service role for AWS Glue. 
* Attach a policy that allows access to Athena, Amazon Simple Storage Service (Amazon S3), and Amazon CloudWatch Logs. For example, you can add [AmazonAthenaFullAccess](https://docs.aws.amazon.com/athena/latest/ug/managed-policies.html#amazonathenafullaccess-managed-policy) and [CloudWatchLogsFullAccess](https://docs.aws.amazon.com/AmazonCloudWatch/latest/logs/iam-identity-based-access-control-cwl.html#managed-policies-cwl) to the role. AmazonAthenaFullAccess allows full access to Athena and includes basic permissions for Amazon S3. CloudWatchLogsFullAccess allows full access to CloudWatch Logs.


![Offline File Compaction](../images/smfs_aws_glue_role.png "Offline File Compaction")

#### Create AWS Glue Job

Next, you need to create the actual AWS Glue Job.

* Navigate to the AWS Glue console.
* Choose the **Jobs** tab under AWS Glue Studio.
* Select **Python Shell script editor**. 
* Choose to *Upload and edit an existing script*. Use the script located in this module [smfs_offine_compaction.py](./smfs_offine_compaction.py)
* Click **Create**.
* Edit the name of the job (as per screenshot below).

![Offline File Compaction](../images/smfs_glue_script_upload.png "Offline File Compaction")



The **Job details** lets you configure the AWS Glue job.

* Select *IAM role* you created earlier on.
* Select *Python 3.9* or the latest available Python version.
* In the same tab, you can also define a number of other configuration options such as **Number of retries** or **Job timeout**. Leave the value as default.

![Offline File Compaction](../images/smfs_glue_job_details.png "Offline File Compaction")


* In the **Advanced properties** section, you can add **Job parameters** to execute the script, as per the example screenshot below.
* Click "Save".

![Offline File Compaction](../images/smfs_glue_job_parameters.png "Offline File Compaction")

* In the **Schedules** tab, you can define the schedule to run the feature store maintenance procedures. For example, the screenshot below shows you how to run the job on a schedule of every 6 hours. For more information, check the [documentation](https://docs.aws.amazon.com/glue/latest/dg/monitor-data-warehouse-schedule.html).

![Offline File Compaction](../images/smfs_glue_schedule.png "Offline File Compaction")

* You can click "Save" and "Run" to start a compaction job.

![Offline File Compaction](../images/smfs_glue_run.png "Offline File Compaction")

* You can also monitor job runs to understand runtime metrics such as completion status, duration, and start time. You can check the CloudWatch logs for the AWS Glue job to check that the procedures run successfully.

![Offline File Compaction](../images/smfs_glue_logs.png "Offline File Compaction")